# Segmenting and Clustering Neighborhoods in Toronto - Part III

In [6]:
import folium 
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np
import pandas as pd
import requests

from geopy.geocoders import Nominatim
from pandas.io.json  import json_normalize 
from sklearn.cluster import KMeans

In [7]:
neighs_and_coords = pd.read_csv('w3_01_list_of_postal_codes_and_coordinates.csv')
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
            len(neighs_and_coords['Borough'].unique()),
            neighs_and_coords.shape[0]
    )
)
neighs_and_coords.shape

The dataframe has 11 boroughs and 103 neighborhoods.


(103, 5)

In [8]:
address    = 'Toronto, CA'
geolocator = Nominatim(user_agent="luiz_alberto_capstone_project")
location  = geolocator.geocode(address)
latitude  = location.latitude
longitude = location.longitude
print('The geographical coordinate of Toronto city are {} {}.'.format(latitude, longitude))

The geographical coordinate of Toronto city are 43.653963 -79.387207.


In [9]:
map_of_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
neighs_and_coords = neighs_and_coords[neighs_and_coords['Borough'].str.contains('Toronto') == True]
for lat, lng, borough, neighborhood in zip(neighs_and_coords['Latitude'], neighs_and_coords['Longitude'], neighs_and_coords['Borough'], neighs_and_coords['Neighborhood']):
    label = '{} {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186c',
        fill_opacity=0.7
    ).add_to(map_of_toronto)
map_of_toronto    

In [10]:
CLIENT_ID = 'RP1P2BKPRKXDHQZIRAFU50GOPAFWLCQDFTK4NJSKIFVQND0J' # your Foursquare ID
CLIENT_SECRET = 'JYR34RL02SLO3CS25WUNAZE1KJC0BAXLSF5AGSZEJGVVONAL' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RP1P2BKPRKXDHQZIRAFU50GOPAFWLCQDFTK4NJSKIFVQND0J
CLIENT_SECRET:JYR34RL02SLO3CS25WUNAZE1KJC0BAXLSF5AGSZEJGVVONAL


In [11]:
neighs_and_coords.loc[37, 'Neighborhood']
neighborhood_latitude = neighs_and_coords.loc[37, 'Latitude'] 
neighborhood_longitude = neighs_and_coords.loc[37, 'Longitude'] 

neighborhood_name = neighs_and_coords.loc[37, 'Neighborhood']

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


In [12]:
# type your answer here
LIMIT  = 100
radius = 500
url    = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url



'https://api.foursquare.com/v2/venues/explore?&client_id=RP1P2BKPRKXDHQZIRAFU50GOPAFWLCQDFTK4NJSKIFVQND0J&client_secret=JYR34RL02SLO3CS25WUNAZE1KJC0BAXLSF5AGSZEJGVVONAL&v=20180605&ll=43.67635739999999,-79.2930312&radius=500&limit=100'

In [13]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5c3bc05c9fb6b7269a137355'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 5,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b8daea1f964a520480833e3',
       'name': 'Grover Pub and Grub',
       'location': {'address': '676 Kingston Rd.',
        'crossStreet': 'at Main St.',
        'lat': 43.679181434941015,
        'lng': -79.29721535878515,
        'labeledLatLng

In [14]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [15]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues) 
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Grover Pub and Grub,Pub,43.679181,-79.297215
1,Starbucks,Coffee Shop,43.678798,-79.298045
2,Upper Beaches,Neighborhood,43.680563,-79.292869
3,Beaches Fitness,Gym / Fitness Center,43.680319,-79.290991
4,Dip 'n Sip,Coffee Shop,43.678897,-79.297745


In [16]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

5 venues were returned by Foursquare.


In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
neighs_and_coords_venues = getNearbyVenues(names=neighs_and_coords['Neighborhood'],
                                   latitudes=neighs_and_coords['Latitude'],
                                   longitudes=neighs_and_coords['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

In [19]:
print(neighs_and_coords_venues.shape)
neighs_and_coords_venues.head()

(1707, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
1,The Beaches,43.676357,-79.293031,Starbucks,43.678798,-79.298045,Coffee Shop
2,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
3,The Beaches,43.676357,-79.293031,Beaches Fitness,43.680319,-79.290991,Gym / Fitness Center
4,The Beaches,43.676357,-79.293031,Dip 'n Sip,43.678897,-79.297745,Coffee Shop


In [20]:
neighs_and_coords_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,54,54,54,54,54,54
"Brockton, Exhibition Place, Parkdale Village",19,19,19,19,19,19
Business reply mail Processing Centre969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",14,14,14,14,14,14


In [21]:
print('There are {} uniques categories.'.format(len(neighs_and_coords_venues['Venue Category'].unique())))

There are 238 uniques categories.


In [22]:
neighs_and_coords_onehot = pd.get_dummies(neighs_and_coords_venues[['Venue Category']], prefix="", prefix_sep="")
neighs_and_coords_onehot['Neighborhood'] = neighs_and_coords_venues['Neighborhood'] 
fixed_columns = [neighs_and_coords_onehot.columns[-1]] + list(neighs_and_coords_onehot.columns[:-1])
neighs_and_coords_onehot = neighs_and_coords_onehot[fixed_columns]
neighs_and_coords_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
neighs_and_coords_onehot.shape

(1707, 238)

In [24]:
neighs_and_coords_grouped = neighs_and_coords_onehot.groupby('Neighborhood').mean().reset_index()
neighs_and_coords_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Adelaide, King, Richmond",0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.01,0.0,0.01
1,Berczy Park,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00
3,Business reply mail Processing Centre969 Eastern,0.058824,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.0,0.0,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00


In [25]:
neighs_and_coords_grouped.shape

(38, 238)

In [26]:
num_top_venues = 5

for hood in neighs_and_coords_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = neighs_and_coords_grouped[neighs_and_coords_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2           Steakhouse  0.04
3      Thai Restaurant  0.04
4  American Restaurant  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1          Restaurant  0.06
2        Cocktail Bar  0.06
3         Cheese Shop  0.04
4  Italian Restaurant  0.04


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0  Breakfast Spot  0.11
1            Café  0.11
2     Coffee Shop  0.11
3    Climbing Gym  0.05
4             Bar  0.05


----Business reply mail Processing Centre969 Eastern----
              venue  freq
0       Yoga Studio  0.06
1     Auto Workshop  0.06
2        Comic Shop  0.06
3              Park  0.06
4  Recording Studio  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0    Airport Lounge  0.14
1   Airport Service 

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighs_and_coords_venues_sorted = pd.DataFrame(columns=columns)
neighs_and_coords_venues_sorted['Neighborhood'] = neighs_and_coords_grouped['Neighborhood']

for ind in np.arange(neighs_and_coords_grouped.shape[0]):
    neighs_and_coords_venues_sorted.iloc[ind, 1:] = return_most_common_venues(neighs_and_coords_grouped.iloc[ind, :], num_top_venues)

neighs_and_coords_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Hotel,Restaurant,Asian Restaurant,Bar,Gym
1,Berczy Park,Coffee Shop,Restaurant,Cocktail Bar,Cheese Shop,Bakery,Italian Restaurant,Beer Bar,Steakhouse,Seafood Restaurant,Pub
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Café,Breakfast Spot,Bar,Stadium,Italian Restaurant,Burrito Place,Climbing Gym,Office,Furniture / Home Store
3,Business reply mail Processing Centre969 Eastern,Yoga Studio,Auto Workshop,Park,Pizza Place,Recording Studio,Restaurant,Burrito Place,Brewery,Skate Park,Smoke Shop
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Lounge,Airport Terminal,Airport Service,Boutique,Sculpture Garden,Plane,Boat or Ferry,Airport Gate,Airport Food Court,Airport


In [41]:
kclusters = 5
neighs_and_coords_grouped_clustering = neighs_and_coords_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(neighs_and_coords_grouped_clustering)
kmeans.labels_

array([2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 3, 2, 2, 2, 2,
       1, 2, 0, 2, 2, 0, 4, 2, 2, 2, 2, 2, 2, 2, 1, 2], dtype=int32)

In [40]:
neighs_and_coords_merged = neighs_and_coords
neighs_and_coords_merged['Cluster Labels'] = kmeans.labels_
neighs_and_coords_merged = neighs_and_coords_merged.join(neighs_and_coords_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
neighs_and_coords_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Coffee Shop,Gym / Fitness Center,Pub,Women's Store,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Yoga Studio,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant
42,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,2,Park,Sushi Restaurant,Board Shop,Brewery,Burger Joint,Sandwich Place,Burrito Place,Pub,Pet Store,Pizza Place
43,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Coworking Space,Seafood Restaurant,Sandwich Place,Cheese Shop
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,2,Bus Line,Park,Swim School,Dim Sum Restaurant,Women's Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197,2,Park,Hotel,Burger Joint,Food & Drink Shop,Dance Studio,Clothing Store,Sandwich Place,Breakfast Spot,Grocery Store,Gym
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,2,Sporting Goods Shop,Coffee Shop,Clothing Store,Salon / Barbershop,Sandwich Place,Diner,Rental Car Location,Furniture / Home Store,Dessert Shop,Chinese Restaurant
47,M4S,Central Toronto,Davisville,43.704324,-79.388790,2,Pizza Place,Dessert Shop,Sandwich Place,Italian Restaurant,Seafood Restaurant,Café,Sushi Restaurant,Coffee Shop,Chinese Restaurant,Diner
48,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160,2,Playground,Gym,Tennis Court,Park,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Diner,Electronics Store
49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049,2,Pub,Coffee Shop,Light Rail Station,Supermarket,Sushi Restaurant,Bagel Shop,Sports Bar,American Restaurant,Pizza Place,Vietnamese Restaurant


In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(neighs_and_coords_merged['Latitude'], neighs_and_coords_merged['Longitude'], neighs_and_coords_merged['Neighborhood'], neighs_and_coords_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [32]:
neighs_and_coords_merged.loc[neighs_and_coords_merged['Cluster Labels'] == 0, neighs_and_coords_merged.columns[[1] + list(range(5, neighs_and_coords_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
65,Central Toronto,0,Coffee Shop,Café,Sandwich Place,Pizza Place,History Museum,Indian Restaurant,Pharmacy,Cosmetics Shop,Pub,Burger Joint
68,Downtown Toronto,0,Airport Lounge,Airport Terminal,Airport Service,Boutique,Sculpture Garden,Plane,Boat or Ferry,Airport Gate,Airport Food Court,Airport


In [33]:
neighs_and_coords_merged.loc[neighs_and_coords_merged['Cluster Labels'] == 1, neighs_and_coords_merged.columns[[1] + list(range(5, neighs_and_coords_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,East Toronto,1,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Coworking Space,Seafood Restaurant,Sandwich Place,Cheese Shop
52,Downtown Toronto,1,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Gay Bar,Restaurant,Burger Joint,Bubble Tea Shop,Café,Fast Food Restaurant,Men's Store
63,Central Toronto,1,Garden,Pool,Music Venue,Women's Store,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
84,West Toronto,1,Coffee Shop,Pizza Place,Café,Sushi Restaurant,Diner,Italian Restaurant,Gourmet Shop,Bookstore,Food,Bar


In [34]:
neighs_and_coords_merged.loc[neighs_and_coords_merged['Cluster Labels'] == 2, neighs_and_coords_merged.columns[[1] + list(range(5, neighs_and_coords_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,East Toronto,2,Coffee Shop,Gym / Fitness Center,Pub,Women's Store,Discount Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
41,East Toronto,2,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Yoga Studio,Cosmetics Shop,Brewery,Bubble Tea Shop,Restaurant
42,East Toronto,2,Park,Sushi Restaurant,Board Shop,Brewery,Burger Joint,Sandwich Place,Burrito Place,Pub,Pet Store,Pizza Place
44,Central Toronto,2,Bus Line,Park,Swim School,Dim Sum Restaurant,Women's Store,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
45,Central Toronto,2,Park,Hotel,Burger Joint,Food & Drink Shop,Dance Studio,Clothing Store,Sandwich Place,Breakfast Spot,Grocery Store,Gym
46,Central Toronto,2,Sporting Goods Shop,Coffee Shop,Clothing Store,Salon / Barbershop,Sandwich Place,Diner,Rental Car Location,Furniture / Home Store,Dessert Shop,Chinese Restaurant
47,Central Toronto,2,Pizza Place,Dessert Shop,Sandwich Place,Italian Restaurant,Seafood Restaurant,Café,Sushi Restaurant,Coffee Shop,Chinese Restaurant,Diner
48,Central Toronto,2,Playground,Gym,Tennis Court,Park,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Diner,Electronics Store
49,Central Toronto,2,Pub,Coffee Shop,Light Rail Station,Supermarket,Sushi Restaurant,Bagel Shop,Sports Bar,American Restaurant,Pizza Place,Vietnamese Restaurant
50,Downtown Toronto,2,Park,Playground,Trail,Diner,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


In [35]:
neighs_and_coords_merged.loc[neighs_and_coords_merged['Cluster Labels'] == 3, neighs_and_coords_merged.columns[[1] + list(range(5, neighs_and_coords_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
57,Downtown Toronto,3,Coffee Shop,Café,Italian Restaurant,Bar,Burger Joint,Chinese Restaurant,Spa,Sandwich Place,Falafel Restaurant,Indian Restaurant


In [36]:
neighs_and_coords_merged.loc[neighs_and_coords_merged['Cluster Labels'] == 4, neighs_and_coords_merged.columns[[1] + list(range(5, neighs_and_coords_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
69,Downtown Toronto,4,Coffee Shop,Restaurant,Café,Italian Restaurant,Seafood Restaurant,Pub,Hotel,Beer Bar,Cocktail Bar,Japanese Restaurant
